# Categorização de Leis a partir das ementas
by Déborah Mesquita



## Etapa 1: carregar o dataset

In [1]:
import pandas as pd

file_path = 'Leis_1992_2016.csv'
leis = pd.read_csv(file_path)
leis.ementa = leis.ementa.astype(str)
leis.ementa = leis.ementa.str.lower()

In [2]:
# Durante a análise descobri que algumas ementas estavam com a primeira palavra duplicada

def clear_first_word_duplicate(ementa):
    words = ementa.split(" ")
    if(len(words) > 2 and words[0] == words[1]):
        return " ".join(words[1:])
    else:
        return ementa
    
leis['ementa'] = leis.apply(lambda row: clear_first_word_duplicate(row['ementa']), axis=1)

leis.ementa = leis.ementa.astype(str)

In [4]:
print("Quantidade de leis: ", len(leis))

Quantidade de leis:  821


In [5]:
leis.head()

,numero,ano,ementa
0,1012,1992,reajusta os vencimentos do pessoal da prefeitu...
1,1013,1992,autoriza ao poder executivo desafetar áreas de...
2,1014,1992,concede pensão financeira à viuva do ex-vice-p...
3,1015,1992,cria cargos (funções) para efeito de concursos...
4,1016,1992,"dá nova redação à alínea ""a"" do artigo 1º da l..."


## Etapa 2: identificar uma teoria de predição
Ao explorar o dataset, foi possível notar que a primeira palavra de cada ementa representa bem uma categorização inicial

In [6]:
from collections import Counter

total_word_frequency = Counter()

for index,row in leis.iterrows():
    ementa = row['ementa']
    for word in ementa.split(" "):
        total_word_frequency[word] += 1

In [7]:
total_word_frequency.most_common()

[('de', 1280),
 ('e', 908),
 ('do', 735),
 ('outras', 590),
 ('dá', 585),
 ('providências.', 483),
 ('a', 475),
 ('o', 426),
 ('da', 385),
 ('municipal', 274),
 ('município', 223),
 ('sobre', 209),
 ('lei', 200),
 ('dispõe', 191),
 ('para', 188),
 ('no', 167),
 ('ipojuca', 158),
 ('dos', 122),
 ('altera', 120),
 ('autoriza', 106),
 ('-', 106),
 ('que', 100),
 ('cria', 94),
 ('poder', 89),
 ('ipojuca,', 89),
 ('ao', 86),
 ('nº', 83),
 ('os', 82),
 ('executivo', 76),
 ('institui', 72),
 ('em', 71),
 ('as', 68),
 ('na', 68),
 ('cargos', 67),
 ('câmara', 64),
 ('exercício', 63),
 ('providencias.', 62),
 ('âmbito', 60),
 ('à', 57),
 ('denomina', 53),
 ('n°', 52),
 ('servidores', 49),
 ('art.', 46),
 ('crédito', 45),
 ('fixa', 45),
 ('plano', 45),
 ('abertura', 41),
 ('providências', 40),
 ('estabelece', 39),
 ('rua', 37),
 ('criação', 36),
 ('neste', 35),
 ('conselho', 35),
 ('distrito', 34),
 ('com', 33),
 ('público', 33),
 ('concede', 31),
 ('especial', 30),
 ('programa', 29),
 ('nome', 2

In [8]:
total_words_reduced = Counter()

for index,row in leis.iterrows():
    ementa = row['ementa']
    for word in ementa.split(" ")[:1]:
        total_words_reduced[word] += 1

In [12]:
print("Quantidade de palavras diferentes utilizando todas as palavras das ementas: ",len(total_word_frequency))
print("Quantiade de palavras diferentes utilizando apenas a primeira palavra: ",len(total_words_reduced))

Quantidade de palavras diferentes utilizando todas as palavras das ementas:  2427
Quantiade de palavras diferentes utilizando apenas a primeira palavra:  67


In [13]:
total_words_reduced.most_common()

[('dispõe', 166),
 ('altera', 108),
 ('autoriza', 93),
 ('cria', 77),
 ('institui', 55),
 ('denomina', 53),
 ('estabelece', 29),
 ('reajusta', 25),
 ('concede', 25),
 ('dá', 17),
 ('fixa', 17),
 ('atribui', 13),
 ('estima', 12),
 ('?', 10),
 ('revoga', 9),
 ('modifica', 8),
 ('desafeta', 7),
 ('disciplina', 6),
 ('institui,', 6),
 ('dota', 5),
 ('abre', 5),
 ('aprova', 5),
 ('reestrutura', 5),
 ('orça', 4),
 ('acrescenta', 4),
 ('lei', 4),
 ('prorroga', 3),
 ('nan', 3),
 ('implanta', 3),
 ('regulamenta', 3),
 ('oferece', 2),
 ('declara', 2),
 ('aumente', 2),
 ('introduz', 2),
 ('insere', 1),
 ('autorizo', 1),
 ('da', 1),
 ('adapta', 1),
 ('autorizao', 1),
 ('eleva', 1),
 ('dispõeõe', 1),
 ('isenta', 1),
 ('reorganiza,', 1),
 ('adequa', 1),
 ('converte,', 1),
 ('define', 1),
 ('criação', 1),
 ('redefine', 1),
 ('vigilância', 1),
 ('designa', 1),
 ('aletera', 1),
 ('aumenta', 1),
 ('equipara', 1),
 ('incentiva', 1),
 ('reduz', 1),
 ('revisa', 1),
 ('doa', 1),
 ('colocação', 1),
 ('comple

### Comentários
Usar apenas a primeira palavra apresentou um bom resultado em classificar as leis; 

Vamos usar as palavras com no mínimo 6 ocorrências e criar uma nova coluna no dataset, representando a classificação das leis

In [15]:
categorias = [x for x,cnt in total_words_reduced.most_common() if cnt > 5 ]

def get_categoria(ementa):
    first_word = ementa.split(" ")[0]
    if (first_word in categorias):
        return first_word
    else:
        return "outra"

leis['categoria_geral'] = leis.apply(lambda row: get_categoria(row['ementa']), axis=1)

In [16]:
import numpy as np
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
from bokeh.charts import Bar, show
output_notebook()

Loading BokehJS ...

In [17]:
p = Bar(leis, 'categoria_geral', values='index', agg='count', title="Quantidade de Leis por categoria", legend='bottom_right')
show(p)

## Categoria 'dispõe'

Podemos ver que a categoria 'dispõe' parece ser a mais geral, portanto vamos começar a criar sub-categorias para este grupo;

Também é possível ver que algumas categorias podem ser combinadas

In [20]:
leis[leis['categoria_geral'] == 'dispõe'].head()

,numero,ano,ementa,categoria_geral
9,1021,1992,dispõe sobre as diretrizes orçamentárias para ...,dispõe
38,1050,1993,dispõe sobre as diretrizes para elaboraçãao do...,dispõe
58,1070,1994,dispõe sobre as diretizes para a elaboração de...,dispõe
70,1083,1995,dispõe sobre a revalidação dos certames públic...,dispõe
72,1085,1995,dispõe sobre as diretrizes para elaboração e e...,dispõe


### Comentários
A quarta palavra das ementas pertencentes a categoria 'dispõe' parece identificar o propósito da lei, vamos usar isto como sub-categoria

In [22]:
sub_categoria_dispoe = Counter()

for index,row in leis.iterrows():
    if (row['categoria_geral'] == 'dispõe'):
        ementa = row['ementa'].split(" ")
        fourth_word = ementa[3]
        sub_categoria_dispoe[fourth_word] += 1

print("Quantidade de sub-categorias: ",len(sub_categoria_dispoe))
sub_categoria_dispoe.most_common()

Quantidade de sub-categorias:  64


[('criação', 27),
 ('plano', 10),
 ('abertura', 9),
 ('revisão', 8),
 ('de', 7),
 ('instituição', 7),
 ('alteração', 7),
 ('estrutura', 6),
 ('diretrizes', 5),
 ('denominação', 5),
 ('reajuste', 4),
 ('vencimentos', 4),
 ('contratação', 3),
 ('aumento', 3),
 ('reestruturação', 3),
 ('reorganização', 2),
 ('interveniência', 2),
 ('verba', 2),
 ('destinação', 2),
 ('isenção', 2),
 ('regime', 2),
 ('do', 2),
 ('conselho', 2),
 ('dos', 2),
 ('diretizes', 1),
 ('revalidação', 1),
 ('feriados', 1),
 ('adicionais', 1),
 ('percentual', 1),
 ('programa', 1),
 ('adequação', 1),
 ('de~crédito', 1),
 ('de^rédito', 1),
 ('direírizes', 1),
 ('introdução', 1),
 ('plantio,', 1),
 ('atendimento', 1),
 ('conselhos', 1),
 ('aplicação', 1),
 ('estatuto', 1),
 ('estruturação', 1),
 ('inclusão', 1),
 ('piano', 1),
 ('municipal', 1),
 ('para', 1),
 ('regulamentação', 1),
 ('nos', 1),
 ('concessão', 1),
 ('ampliação', 1),
 ('e', 1),
 ('criação,', 1),
 ('obrigatoriedade', 1),
 ('proibição', 1),
 ('licenciament

### Observações
A categoria 'de' ficou com um número considerável de ocorrências, vamos remover as stop_words e usar a próxima palavra da ementa nesses casos

In [23]:
sub_categoria_dispoe = Counter()
stop_words = ['a', 'o' , 'de', 'as', 'do', 'e', 'os' , 'dos', 'no', 'da', 'para']

for index,row in leis.iterrows():
    if (row['categoria_geral'] == 'dispõe'):
        ementa = row['ementa'].split(" ")
        fourth_word = ementa[3]
        fifth_word = ementa[4]
        if (fourth_word in stop_words):
            sub_categoria_dispoe[fifth_word] += 1
        else:
            sub_categoria_dispoe[fourth_word] += 1
            
print("Quantidade de sub-categorias: ",len(sub_categoria_dispoe))       
sub_categoria_dispoe.most_common()

Quantidade de sub-categorias:  65


[('criação', 27),
 ('plano', 10),
 ('abertura', 9),
 ('revisão', 8),
 ('instituição', 7),
 ('alteração', 7),
 ('estrutura', 6),
 ('vencimentos', 6),
 ('diretrizes', 5),
 ('crédito', 5),
 ('denominação', 5),
 ('reajuste', 4),
 ('contratação', 3),
 ('aumento', 3),
 ('reestruturação', 3),
 ('reorganização', 2),
 ('interveniência', 2),
 ('verba', 2),
 ('destinação', 2),
 ('isenção', 2),
 ('regime', 2),
 ('conselho', 2),
 ('doação', 2),
 ('diretizes', 1),
 ('revalidação', 1),
 ('feriados', 1),
 ('adicionais', 1),
 ('percentual', 1),
 ('programa', 1),
 ('créditos', 1),
 ('adequação', 1),
 ('de~crédito', 1),
 ('de^rédito', 1),
 ('direírizes', 1),
 ('introdução', 1),
 ('plantio,', 1),
 ('atendimento', 1),
 ('diagnóstico', 1),
 ('conselhos', 1),
 ('aplicação', 1),
 ('estatuto', 1),
 ('estruturação', 1),
 ('inclusão', 1),
 ('piano', 1),
 ('municipal', 1),
 ('regulamentação', 1),
 ('nos', 1),
 ('concessão', 1),
 ('ampliação', 1),
 ('competência', 1),
 ('remanejamento', 1),
 ('criação,', 1),
 ('ob

### Comentários
Agora sim estamos com um resultado legal, vamos usar como sub-categorias as palavras com mais de 2 ocorrências

In [24]:
sub_categorias_dispoe = [x for x,cnt in sub_categoria_dispoe.most_common() if cnt > 2 ]

def get_sub_categoria_dispoe(ementa):
    if (len(ementa.split(" ")) > 3):
        fourth_word = ementa.split(" ")[3]
        fifth_word = ementa.split(" ")[4]
        if (fourth_word in sub_categorias_dispoe):
            return fourth_word
        elif (fifth_word in sub_categorias_dispoe):
            return fifth_word
        else:
            return "outra"
    else:
        return "outra"

leis['sub_categoria'] = leis.apply(lambda row: get_sub_categoria_dispoe(row['ementa']), axis=1)

In [26]:
p = Bar(leis, 'categoria_geral', values='index', agg='count', title="Quantidade de Leis por sub-categoria", stack='sub_categoria')
show(p)

## Categoria 'modifica'
Vamos investigar se as categorias 'altera' e 'modifica' podem ser combinadas

In [28]:
print(leis[leis['categoria_geral'] == 'altera']['ementa'].head())
print(leis[leis['categoria_geral'] == 'modifica']['ementa'].head())

89     altera o art.2 da lei municipal 1046/93 e dá o...
130    altera classificação de cargos comissionados d...
134    altera a lei n° 1.100/95 e dá outras providenc...
135    altera vantagens dos professores do ensino de ...
153    altera simbologia de cargo de provimento em co...
Name: ementa, dtype: object
353    modifica e renumera a lei municipal n° 1.139/9...
355    modifica a lei municipal n.° 1.312/02 e dá out...
392    modifica os deveres e atribuições dos cargos d...
402    modifica o anexo único da lei 1414/2005, crian...
415    modifica o artigo 4° da lei 1337/2002 e dá out...
Name: ementa, dtype: object


### Comentários
As duas parecem alterar Leis, vamos combiná-las

In [29]:
def change_category_label(to_change, old_category, new_category):
    if(old_category == to_change):
        return new_category
    else:
        return old_category
    
leis['categoria_geral'] = leis.apply(lambda row: change_category_label('modifica', row['categoria_geral'],'altera'), axis=1)

In [31]:
p = Bar(leis, 'categoria_geral', values='index', agg='count', title="Quantidade de Leis por sub-categoria", stack='sub_categoria')
show(p)

## Categoria 'altera'
Aparentemente todas as leis com 'altera' na primeira palavra da ementa alteram uma lei, vamos investigar isso

In [32]:
def get_counter_with_category(category, keyword, index_word):
    counter = Counter()
    for index,row in leis.iterrows():
        if (row[category] == keyword):
            ementa = row['ementa'].split(" ")
            if(len(ementa) > index_word):
                word = ementa[index_word]
                counter[word] += 1
    return counter
    
sub_categoria_altera = get_counter_with_category('categoria_geral','altera',2)
               
sub_categoria_altera.most_common()

[('lei', 31),
 ('anexo', 15),
 ('da', 11),
 ('art.', 10),
 ('artigo', 6),
 ('redação', 5),
 ('§', 4),
 ('composição', 3),
 ('arts.', 3),
 ('estrutura', 3),
 ('§§', 3),
 ('de', 2),
 ('dos', 2),
 ('inciso', 2),
 ('acrescenta', 2),
 ('incisos', 2),
 ('art.2', 1),
 ('introduz', 1),
 ('valor', 1),
 ('renumera', 1),
 ('deveres', 1),
 ('artigos', 1),
 ('anexos', 1),
 ('disposições', 1),
 ('alíquota', 1),
 ('att', 1),
 ('parágrafo', 1),
 ('suprime', 1)]

### Comentários
Confirmado, as leis com 'altera' sempre se referem a leis

Vamos mudar a label de 'altera' para 'altera lei', assim deixamos mais claro; outra obervação é que nesse caso não precisamos de sub-categoria, então substituimos esse campo com '-'

In [33]:
def change_category_label(to_change, old_category, new_category):
    if(old_category == to_change):
        return new_category
    else:
        return old_category
    
def change_sub_category_label(to_change, category,old_sub_category, new_sub_category):
    if (category == to_change):
        return new_sub_category
    else:
        return old_sub_category
    
leis['categoria_geral'] = leis.apply(lambda row: change_category_label('altera', row['categoria_geral'],'altera lei'), axis=1)
leis['sub_categoria'] = leis.apply(lambda row: change_sub_category_label('altera lei',row['categoria_geral'],row['sub_categoria'],'-'), axis=1)

In [34]:
p = Bar(leis, 'categoria_geral', values='index', agg='count', title="Quantidade de Leis por sub-categoria", stack='sub_categoria')
show(p)

## Categoria 'autoriza'
Todas as leis com 'autoriza' se referem ao poder executiva, vamos mudar a label para 'autoriza poder executivo'

In [36]:
leis[leis['categoria_geral'] == 'autoriza'].head()

,numero,ano,ementa,categoria_geral,sub_categoria
1,1013,1992,autoriza ao poder executivo desafetar áreas de...,autoriza,outra
11,1023,1992,autoriza ao poder executivo desmembrar e desaf...,autoriza,outra
13,1025,1992,autoriza a abertura de crédito especial e dá o...,autoriza,crédito
22,1034,1993,"autoriza a dispensa de multa, juros e redução ...",autoriza,outra
23,1035,1993,autoriza os chefes dos poderes executivos e le...,autoriza,outra


In [37]:
sub_categoria_autoriza = get_counter_with_category('categoria_geral','autoriza',2)
               
sub_categoria_autoriza.most_common()

[('poder', 44),
 ('abertura', 13),
 ('de', 8),
 ('reajuste', 6),
 ('contratação', 5),
 ('chefe', 3),
 ('município', 3),
 ('cessão', 2),
 ('dispensa', 1),
 ('chefes', 1),
 ('concessão', 1),
 ('poderes', 1),
 ('pbcfer', 1),
 ('da', 1),
 ('pagamento', 1),
 ('câmara', 1),
 ('celebração', 1)]

### Observações
Podemos ver que 'poder' tem quase todas as ocorrências (referente ao poder executivo); vamos mudar a label para 'autoriza poder executivo'

In [38]:
leis['categoria_geral'] = leis.apply(lambda row: change_category_label('autoriza', row['categoria_geral'],'autoriza poder executivo'), axis=1)

In [39]:
p = Bar(leis, 'categoria_geral', values='index', agg='count', title="Quantidade de Leis por sub-categoria", stack='sub_categoria')
show(p)

## Categoria 'cria'
Esta categoria pode ter sub-categorizas interessantes, vamos investigar isso

In [40]:
leis[leis['categoria_geral'] == 'cria'].head()

,numero,ano,ementa,categoria_geral,sub_categoria
3,1015,1992,cria cargos (funções) para efeito de concursos...,cria,outra
21,1033,1993,cria cargos de assessores dos secretarios muni...,cria,outra
26,1038,1993,"cria o departamento de agricultura, cargos com...",cria,outra
35,1047,1993,cria cargos em comissão e dá outras providências,cria,outra
52,1064,1994,cria o conselho municipal de defesa do meio am...,cria,outra


In [41]:
sub_categoria_cria = Counter()

for index,row in leis.iterrows():
    if (row['categoria_geral'] == 'cria'):
        ementa = row['ementa'].split(" ")
        fourth_word = ementa[1]
        fifth_word = ementa[2]
        if (fourth_word in stop_words):
            sub_categoria_cria[fifth_word] += 1
        else:
            sub_categoria_cria[fourth_word] += 1
            
        
sub_categoria_cria.most_common()

[('cargos', 25),
 ('nome', 17),
 ('conselho', 9),
 ('secretaria', 4),
 ('cargo', 3),
 ('fundo', 3),
 ('plano', 2),
 ('sistema', 2),
 ('departamento', 1),
 ('diretoria', 1),
 ('medalha', 1),
 ('trofeu', 1),
 ('duas', 1),
 ('guarda', 1),
 ('programa', 1),
 ('coordenadoria', 1),
 ('âmbito', 1),
 ('reserva', 1),
 ('02', 1),
 ('2', 1)]

### Comentários
Vamos utilizar as sub-categorias com mais de 3 ocorrências

In [49]:
categorias_cria = [x for x,cnt in sub_categoria_cria.most_common() if cnt > 3 ]

def get_sub_categoria_cria(ementa):
    ementa = ementa.split(" ")
    fourth_word = ementa[1]
    fifth_word = ementa[2]
    if (fourth_word in categorias_cria):
        return fourth_word
    elif (fifth_word in categorias_cria):
        return fifth_word
    else:
        return "outra"
      
def change_sub_category_label_cria(to_change, category,old_sub_category, ementa):
    if (category == to_change):
        return get_sub_categoria_cria(ementa)
    else:
        return old_sub_category
    
leis['sub_categoria'] = leis.apply(lambda row: change_sub_category_label_cria('cria',row['categoria_geral'],row['sub_categoria'],row['ementa']), axis=1)

In [50]:
leis[leis['categoria_geral'] == 'cria'].head()

,numero,ano,ementa,categoria_geral,sub_categoria
3,1015,1992,cria cargos (funções) para efeito de concursos...,cria,cargos
21,1033,1993,cria cargos de assessores dos secretarios muni...,cria,cargos
26,1038,1993,"cria o departamento de agricultura, cargos com...",cria,outra
35,1047,1993,cria cargos em comissão e dá outras providências,cria,cargos
52,1064,1994,cria o conselho municipal de defesa do meio am...,cria,conselho


In [51]:
p = Bar(leis, 'categoria_geral', values='index', agg='count', title="Quantidade de Leis por sub-categoria", stack='sub_categoria')
show(p)

## Categoria 'institui'
Primeiro vamos combinar 'institui,' e 'institui'

In [52]:
leis['categoria_geral'] = leis.apply(lambda row: change_category_label('institui,', row['categoria_geral'],'institui'), axis=1)

In [53]:
p = Bar(leis, 'categoria_geral', values='index', agg='count', title="Quantidade de Leis por sub-categoria", stack='sub_categoria')
show(p)

In [54]:
leis[leis['categoria_geral'] == 'institui'].head()

,numero,ano,ementa,categoria_geral,sub_categoria
87,1100,1995,institui a gratificação de produtividade fisca...,institui,outra
125,1138,1997,institui o fundo municipal de saúde e dá outra...,institui,outra
168,1181,1998,institui o código tributário do município do i...,institui,outra
184,1197,1999,institui o programa de garantia de renda mínim...,institui,outra
185,1198,1999,institui a gratificação adicional de risco de ...,institui,outra


In [55]:
sub_categoria_institui = get_counter_with_category('categoria_geral','institui',2)
sub_categoria_institui.most_common()

[('programa', 11),
 ('fundo', 5),
 ('código', 5),
 ('dia', 4),
 ('âmbito', 4),
 ('"dia', 3),
 ('gratificação', 2),
 ('rede', 2),
 ('município,', 2),
 ('semana', 2),
 ('auxílio-transporte', 2),
 ('e', 1),
 ('para', 1),
 ('plano', 1),
 ('contribuição', 1),
 ('percentual', 1),
 ('calendário', 1),
 ('verba', 1),
 ('de', 1),
 ('auxilio', 1),
 ('gratificações', 1),
 ('"prêmio', 1),
 ('sistema', 1),
 ('auxílio-saúde', 1),
 ('regulamenta', 1),
 ('comissão', 1),
 ('suprimento', 1),
 ('tratamento', 1),
 ('“semana', 1),
 ('município', 1)]

In [56]:
categorias_institui = [x for x,cnt in sub_categoria_institui.most_common() if cnt > 3 ]

def get_sub_categoria_institui(ementa):
    ementa = ementa.split(" ")
    word = ementa[2]
    if (word in categorias_institui):
        return word
    elif (word == '"dia'):
        return 'dia'
    else:
        return "outra"
      
def change_sub_category_label_institui(to_change, category,old_sub_category, ementa):
    if (category == to_change):
        return get_sub_categoria_institui(ementa)
    else:
        return old_sub_category
    
leis['sub_categoria'] = leis.apply(lambda row: change_sub_category_label_institui('institui',row['categoria_geral'],row['sub_categoria'],row['ementa']), axis=1)

In [57]:
leis[leis['categoria_geral'] == 'institui'].head()

,numero,ano,ementa,categoria_geral,sub_categoria
87,1100,1995,institui a gratificação de produtividade fisca...,institui,outra
125,1138,1997,institui o fundo municipal de saúde e dá outra...,institui,fundo
168,1181,1998,institui o código tributário do município do i...,institui,código
184,1197,1999,institui o programa de garantia de renda mínim...,institui,programa
185,1198,1999,institui a gratificação adicional de risco de ...,institui,outra


### Categorias 'estima', 'fixa', 'estabelece'
Vamos mudar estas categorias para 'receitas e despesas'

In [59]:
print(leis[leis['categoria_geral'] == 'estima']['ementa'].head())
print(leis[leis['categoria_geral'] == 'fixa']['ementa'].head())
print(leis[leis['categoria_geral'] == 'estabelece']['ementa'].head())

136    estima a receita e fixa a despesa para o exerc...
187    estima a reita e fixa a despesa para o exercíc...
219    estima a receita e fixa despesas para o exercí...
321    estima a receita e fixa a despesa da prefeitur...
351    estima a receita e fixa a despesa da prefeitur...
Name: ementa, dtype: object
49     fixa o salário mínimo em urv e dá outras provi...
69     fixa o valor do salário mínimo para os funcion...
170    fixa o subsídio dos vereadores deste município...
171    fixa os subsídios do prefeito, do viceprefeito...
210    fixa os subsídios do prefeito, do vice-prefeit...
Name: ementa, dtype: object
123    estabelece as diretrizes orçamen tárias do nün...
155    estabelece as diretrizes orçamentárias do muni...
206    estabelece as diretrizes orçamentarias para o ...
214    estabelece o chancfélamento do ingressos para ...
215    estabelece o subsfi|uto tributário do iss e dá...
Name: ementa, dtype: object


In [60]:
leis['categoria_geral'] = leis.apply(lambda row: change_category_label('estima', row['categoria_geral'],'receitas e despesas'), axis=1)
leis['categoria_geral'] = leis.apply(lambda row: change_category_label('fixa', row['categoria_geral'],'receitas e despesas'), axis=1)
leis['categoria_geral'] = leis.apply(lambda row: change_category_label('estabelece', row['categoria_geral'],'receitas e despesas'), axis=1)

In [61]:
leis[leis['categoria_geral'] == 'receitas e despesas'].head()

,numero,ano,ementa,categoria_geral,sub_categoria
49,1061,1994,fixa o salário mínimo em urv e dá outras provi...,receitas e despesas,outra
69,1082,1995,fixa o valor do salário mínimo para os funcion...,receitas e despesas,outra
123,1136,1997,estabelece as diretrizes orçamen tárias do nün...,receitas e despesas,outra
136,1149,1997,estima a receita e fixa a despesa para o exerc...,receitas e despesas,outra
155,1168,1998,estabelece as diretrizes orçamentárias do muni...,receitas e despesas,outra


In [62]:
categorias_receitas_despesas = ['estima', 'fixa', 'estabelece']

def get_sub_categoria_receita_despesas(ementa):
    ementa = ementa.split(" ")
    word = ementa[0]
    return word
      
def change_sub_category_label_cria(to_change, category,old_sub_category, ementa):
    if (category == to_change):
        return get_sub_categoria_receita_despesas(ementa)
    else:
        return old_sub_category
    
leis['sub_categoria'] = leis.apply(lambda row: change_sub_category_label_cria('receitas e despesas',row['categoria_geral'],row['sub_categoria'],row['ementa']), axis=1)

In [63]:
leis[leis['categoria_geral'] == 'receitas e despesas'].head()

,numero,ano,ementa,categoria_geral,sub_categoria
49,1061,1994,fixa o salário mínimo em urv e dá outras provi...,receitas e despesas,fixa
69,1082,1995,fixa o valor do salário mínimo para os funcion...,receitas e despesas,fixa
123,1136,1997,estabelece as diretrizes orçamen tárias do nün...,receitas e despesas,estabelece
136,1149,1997,estima a receita e fixa a despesa para o exerc...,receitas e despesas,estima
155,1168,1998,estabelece as diretrizes orçamentárias do muni...,receitas e despesas,estabelece


## Categoria 'atribui'
Vamos combinar a categoria 'atribui' com a categoria 'denomina'

In [65]:
print(leis[leis['categoria_geral'] == 'atribui']['ementa'].head())
print(leis[leis['categoria_geral'] == 'denomina']['ementa'].head())

77    atribui denominação a bem público municipal e ...
81    atribui a bem público municipal e da outras pr...
93    atribui denominação a bem público e dá outras ...
94    atribui denominação a bem público e dá outras ...
95    atribui denominação a bem público e dá outras ...
Name: ementa, dtype: object
78     denomina rua josé hipolito monteiro a rua atua...
79     denomina rua theodomiro josé camargo silva a r...
80     denomina rua mário luiz cavalcanti a atualment...
164    denomina artéria pública e dá outras providênc...
174    denomina artéria pública e dá outras providênc...
Name: ementa, dtype: object


In [66]:
leis['categoria_geral'] = leis.apply(lambda row: change_category_label('atribui', row['categoria_geral'],'denomina'), axis=1)

## Categorias 'disicplina', 'dá', 'concede'
Podemos combinar estas categorias

In [67]:
print(leis[leis['categoria_geral'] == 'disciplina']['ementa'])

251    disciplina a concessão de incentivos fiscais a...
391    disciplina a concessão de incentivos fiscais a...
479    disciplina a concessão de incentivos fiscais p...
489    disciplina a permissão de serviços de transpor...
545    disciplina a obrigatoriedade e a utilização do...
614    disciplina o regime de extinção dos empregos p...
Name: ementa, dtype: object


In [68]:
leis['categoria_geral'] = leis.apply(lambda row: change_category_label('disciplina', row['categoria_geral'],'concede'), axis=1)

In [71]:
leis[leis['categoria_geral'] == 'dá']

,numero,ano,ementa,categoria_geral,sub_categoria
4,1016,1992,"dá nova redação à alínea ""a"" do artigo 1º da l...",dá,outra
10,1022,1992,dá nova redação no artigo 1º da lei 1009 de 1....,dá,outra
24,1036,1993,dá designação a logradouros públicos - rua hil...,dá,outra
29,1041,1993,dá designação a logradouros av. gilvan leonico...,dá,outra
30,1042,1993,dá designação ao estádio municipal de nossa se...,dá,outra
31,1043,1993,dá designação a bem público centro de saúde ve...,dá,outra
43,1055,1993,dá nova redação ao art.2 da lei 1052/93 de 10/...,dá,outra
50,1062,1994,dá nova redação do art. 1 da lei 1048/93 de 05...,dá,outra
75,1088,1995,dá nova redação aos arts. 72. da lei n. 1070/9...,dá,outra
282,1296,2001,dá nova redação a dispositivos da lei n.° 1279...,dá,outra


In [72]:
def change_category_da(to_change, old_category, ementa):
    if(old_category == to_change):
        ementa = ementa.split(" ")
        if(ementa[2] == 'nome' or ementa[1] == 'designação'):
            return 'denomina'
        elif(ementa[2] == 'redação'):
            return 'altera lei'
        else:
            return 'outra'
    else:
        return old_category
    
    
leis['categoria_geral'] = leis.apply(lambda row: change_category_da('dá', row['categoria_geral'],row['ementa']), axis=1)
leis['sub_categoria'] = leis.apply(lambda row: change_sub_category_label('altera lei',row['categoria_geral'],row['sub_categoria'],'-'), axis=1)

## Categoria 'denomina'

In [77]:
leis[leis['categoria_geral'] == 'denomina'].head()

,numero,ano,ementa,categoria_geral,sub_categoria
24,1036,1993,dá designação a logradouros públicos - rua hil...,denomina,outra
29,1041,1993,dá designação a logradouros av. gilvan leonico...,denomina,outra
30,1042,1993,dá designação ao estádio municipal de nossa se...,denomina,outra
31,1043,1993,dá designação a bem público centro de saúde ve...,denomina,outra
77,1090,1995,atribui denominação a bem público municipal e ...,denomina,outra


In [75]:
sub_categoria_denomina = Counter()

for index,row in leis.iterrows():
    if (row['categoria_geral'] == 'denomina'):
        ementa = row['ementa'].split(" ")
        second_word = ementa[1]
        third_word = ementa[2]
        if (second_word in stop_words):
            sub_categoria_denomina[third_word] += 1
        else:
            if(second_word == 'designação' or second_word == 'denominação'):
                word = ementa[3]
                sub_categoria_denomina[word] +=1
            else:
                sub_categoria_denomina[second_word] += 1
            
        
sub_categoria_denomina.most_common()

[('bem', 11),
 ('rua', 9),
 ('nome', 9),
 ('artéria', 3),
 ('praça', 3),
 ('ruas', 3),
 ('centro', 3),
 ('escola', 3),
 ('logradouros', 2),
 ('estádio', 2),
 ('rodovia', 2),
 ('biblioteca', 2),
 ('travessa', 2),
 ('quadra', 1),
 ('banda', 1),
 ('avenida', 1),
 ('hospital', 1),
 ("'conjunto", 1),
 ('"denomina', 1),
 ("'praça", 1),
 ('av.', 1),
 ('anexos', 1),
 ('ruas,', 1),
 ('creche', 1),
 ('por', 1),
 ('adnilson', 1),
 ('professora', 1),
 ('"rodovia', 1),
 ('amaro', 1),
 ('misael', 1),
 ('jonas', 1),
 ('josé', 1),
 ('unidade', 1),
 ('jorge', 1)]

In [78]:
categorias_denomina = [x for x,cnt in sub_categoria_denomina.most_common() if cnt > 2 ]

def get_sub_categoria_denomina(ementa):
    ementa = ementa.split(" ")
    second_word = ementa[1]
    fifth_word = ementa[2]
    if (second_word in categorias_denomina):
        return second_word
    else:
        if(second_word == 'designação' or second_word == 'denominação'):
            word = ementa[3]
            return word
        else:
            return 'outra'
      
    
def change_sub_category_label_denomina(to_change, category,old_sub_category, ementa):
    if (category == to_change):
        return get_sub_categoria_denomina(ementa)
    else:
        return old_sub_category
    
leis['sub_categoria'] = leis.apply(lambda row: change_sub_category_label_denomina('denomina',row['categoria_geral'],row['sub_categoria'],row['ementa']), axis=1)

In [79]:
leis[leis['categoria_geral'] == 'denomina'].head()

,numero,ano,ementa,categoria_geral,sub_categoria
24,1036,1993,dá designação a logradouros públicos - rua hil...,denomina,logradouros
29,1041,1993,dá designação a logradouros av. gilvan leonico...,denomina,logradouros
30,1042,1993,dá designação ao estádio municipal de nossa se...,denomina,estádio
31,1043,1993,dá designação a bem público centro de saúde ve...,denomina,bem
77,1090,1995,atribui denominação a bem público municipal e ...,denomina,bem


In [86]:
p = Bar(leis, 'categoria_geral', values='index', agg='count', title="Quantidade de Leis por categoria")
show(p)

In [121]:
p = Bar(leis[leis['categoria_geral']=='cria'], 'sub_categoria', values='index', agg='count', title="Leis da categoria 'cria'", stack='sub_categoria')
show(p)

## Categoria 'outra'
Esta categoria pode ser refinada, mas exige uma análise mais elaborda, então só vamos fazer isso no futuro

In [81]:
leis[leis['categoria_geral'] == 'outra'].head()

,numero,ano,ementa,categoria_geral,sub_categoria
5,1017,1992,insere parágrafo único do art. 1º da lei n. 97...,outra,outra
6,1018,1992,autorizo o poder executivo a firmar acordo de ...,outra,outra
15,1027,1992,oferece incentivos sobre iptu e dá outras prov...,outra,outra
16,1028,1992,da denominação a prédios e logradouros públicos.,outra,outra
18,1030,1992,orça a receita e fixa a despesa do município p...,outra,outra


## Considerações finais
Os resultados foram satisfatórios para uma classificação simples, agora com este dataset com labels já é possível treinar um modelo para classificar as Leis de 2017

Em um segundo momento vamos trabalhar outros aspectos do processamento de linguagem natural para classificar as Leis da categoria 'outra'

In [84]:
leis['ementa'] = leis.apply(lambda row: (row['ementa']).capitalize(), axis=1)
leis.to_csv('Leis_2016_1992_categorizadas.csv')